# Preamble

Singapore is a country in Southeast Asia with a parliamentary democracy. It has a single chamber parliament comprising _.

In [1]:
import pandas as pd
import requests
import re
import datetime
import json
import sqlite3

In [9]:
PAGE = "https://www.parliament.gov.sg/parliamentary-business/votes-and-proceedings?parliament=14&fromDate=&toDate=&page=1&pageSize=100"

dates_source = requests.get(PAGE).text
date_pattern = r"Sitting on (\d{1,2}\s\w+\s\d{4})"
dates = re.findall(date_pattern, dates_source)
# dates

In [12]:
dates

['20 October 2022',
 '5 October 2022',
 '4 October 2022',
 '3 October 2022',
 '13 September 2022',
 '12 September 2022',
 '2 August 2022',
 '1 August 2022',
 '5 July 2022',
 '4 July 2022',
 '9 May 2022',
 '5 April 2022',
 '4 April 2022',
 '11 March 2022',
 '10 March 2022',
 '9 March 2022',
 '8 March 2022',
 '7 March 2022',
 '4 March 2022',
 '3 March 2022',
 '2 March 2022',
 '1 March 2022',
 '28 February 2022',
 '18 February 2022',
 '15 February 2022',
 '14 February 2022',
 '12 January 2022',
 '11 January 2022',
 '10 January 2022',
 '3 November 2021',
 '2 November 2021',
 '1 November 2021',
 '5 October 2021',
 '4 October 2021',
 '14 September 2021',
 '13 September 2021',
 '3 August 2021',
 '2 August 2021',
 '27 July 2021',
 '26 July 2021',
 '6 July 2021',
 '5 July 2021',
 '11 May 2021',
 '10 May 2021',
 '5 April 2021',
 '8 March 2021',
 '5 March 2021',
 '4 March 2021',
 '3 March 2021',
 '2 March 2021',
 '1 March 2021',
 '26 February 2021',
 '25 February 2021',
 '24 February 2021',
 '16 

In [17]:
dates_df = (pd.DataFrame(dates, columns=['sitting_dates'])
            .assign(sitting_date_dt_ts=lambda x: pd.to_datetime(x.sitting_dates),
                    sitting_date_dt_str=lambda x: [datetime.datetime.strftime(x, "%Y-%m-%d") for x in
                                                   x.sitting_date_dt_ts],
                    param_date=lambda x: [datetime.datetime.strftime(x, "%d-%m-%Y") for x in x.sitting_date_dt_ts]
                    )
            )

In [19]:
# dates_df

In [10]:
def load_raw_data():
    headers = {
        'Accept': 'application/json, text/plain, */*',
        'Accept-Language': 'en-GB,en-US;q=0.9,en;q=0.8',
        'Connection': 'keep-alive',
        'Origin': 'https://sprs.parl.gov.sg',
        'Referer': 'https://sprs.parl.gov.sg/search/',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-origin',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36',
        'sec-ch-ua': '"Google Chrome";v="107", "Chromium";v="107", "Not=A?Brand";v="24"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"macOS"',
    }

    json_data = {
        'headers': {
            'normalizedNames': {},
            'lazyUpdate': None,
        },
    }

    con = sqlite3.connect("/Users/whkoh/git-repos/sg-parl/data.db")
    dates_df1 = dates_df[:10]

    # for row in dates_df1.itertuples():
    #     print(row.param_date)

    for row in dates_df.itertuples():
        sitting_date_parsed = datetime.datetime.strptime(row.param_date, "%d-%m-%Y").strftime("%Y-%m-%d")
        params = {'sittingDate': row.param_date}
        print(row.param_date, sitting_date_parsed)
        response = requests.post('https://sprs.parl.gov.sg/search/getHansardReport/',
                                 params=params, headers=headers, json=json_data)
        raw_json = json.dumps(response.json())
        slim_json = re.sub(r"<img.*?>", "", raw_json)
        cur = con.cursor()
        cur.execute("""
               INSERT INTO json_raw
                 (sitting_date, raw)
               VALUES
                 (?, ?)
        """, (sitting_date_parsed, slim_json))
        con.commit()


20-10-2022 2022-10-20
05-10-2022 2022-10-05
04-10-2022 2022-10-04
03-10-2022 2022-10-03
13-09-2022 2022-09-13
12-09-2022 2022-09-12
02-08-2022 2022-08-02
01-08-2022 2022-08-01
05-07-2022 2022-07-05
04-07-2022 2022-07-04
09-05-2022 2022-05-09
05-04-2022 2022-04-05
04-04-2022 2022-04-04
11-03-2022 2022-03-11
10-03-2022 2022-03-10
09-03-2022 2022-03-09
08-03-2022 2022-03-08
07-03-2022 2022-03-07
04-03-2022 2022-03-04
03-03-2022 2022-03-03
02-03-2022 2022-03-02
01-03-2022 2022-03-01
28-02-2022 2022-02-28
18-02-2022 2022-02-18
15-02-2022 2022-02-15
14-02-2022 2022-02-14
12-01-2022 2022-01-12
11-01-2022 2022-01-11
10-01-2022 2022-01-10
03-11-2021 2021-11-03
02-11-2021 2021-11-02
01-11-2021 2021-11-01
05-10-2021 2021-10-05
04-10-2021 2021-10-04
14-09-2021 2021-09-14
13-09-2021 2021-09-13
03-08-2021 2021-08-03
02-08-2021 2021-08-02
27-07-2021 2021-07-27
26-07-2021 2021-07-26
06-07-2021 2021-07-06
05-07-2021 2021-07-05
11-05-2021 2021-05-11
10-05-2021 2021-05-10
05-04-2021 2021-04-05
08-03-2021

In [20]:
def test_db():
    f = "/Users/whkoh/git-repos/sg-parl/data.db.xz"
    import lzma
    import tempfile
    s = lzma.LZMAFile(f)
    tf = tempfile.NamedTemporaryFile()
    tf.write(s.read())
    con1 = sqlite3.connect(tf.name)
    cur1 = con1.cursor()
    sql = r"""SELECT t.* FROM json_raw t LIMIT 501;"""
    res = cur1.execute(sql)
    print(res.fetchone())

In [66]:
def load_attendance():
    conn = sqlite3.connect("/Users/whkoh/git-repos/sg-parl/data.db")
    cur = conn.cursor()
    sql = """select sitting_date, json_extract(raw, '$.attendanceList') from json_raw t
    -- where sitting_date = '2022-10-20'
    ;"""
    r = cur.execute(sql)
    days = r.fetchall()
    overall_attendance_df = pd.DataFrame()
    member_constituency_df = pd.DataFrame()
    for day in days:
        attendance_df = (pd.DataFrame(json.loads(day[1]))
        .assign(sitting_date=pd.to_datetime(day[0], format="%Y-%m-%d"),
                ret=lambda x: [re.findall(r"(.*?)\s\((.*?)\)", y)[0] for y in x.mpName],
                member_name=lambda x: [y[0] for y in x.ret],
                constituency=lambda x: [y[1].split('(')[-1] for y in x.ret]
                )
        .rename(columns={'attendance': 'attended'})
        )
        row_mp_constituency = attendance_df[['member_name', 'constituency']]
        day_attendance = attendance_df[['member_name', 'attended', 'sitting_date']]
        overall_attendance_df = pd.concat([overall_attendance_df, day_attendance])
        member_constituency_df = pd.concat([row_mp_constituency, member_constituency_df])

    member_constituency_df.drop_duplicates(inplace=True)
    overall_attendance_df.to_sql('attendance', conn, if_exists='replace', index=False)
    member_constituency_df.to_sql('member', conn, if_exists='replace', index=False)
    conn.close()

load_attendance()

In [55]:
df_attendance = (pd.DataFrame(response.json().get('attendanceList'))
                 .assign(ret=lambda x: [re.findall(r"(.*?)\s\((.*?)\)", y)[0] for y in x.mpName],
                         mp=lambda x: [y[0] for y in x.ret],
                         constituency=lambda x: [y[1] for y in x.ret]
                         )
                 )

In [ ]:
df_attendance.head(30)

In [81]:
def load_section():
    conn = sqlite3.connect("/Users/whkoh/git-repos/sg-parl/data.db")
    cur = conn.cursor()
    sql = """select sitting_date, json_extract(raw, '$.takesSectionVOList') from json_raw t
    where sitting_date = '2022-10-20'
    ;"""
    r = cur.execute(sql)
    i, j = r.fetchone()
    df = (pd.DataFrame.from_records(json.loads(j))
          .assign(sitting_date=i)
          .astype(str)
          .to_sql('item', con=conn, if_exists='append', index=False)
          )

    return

load_section()


In [23]:

b = 'srr'

for c, d in zip(a, **b):
    print(c, d)

TypeError: zip() argument after ** must be a mapping, not str